<a href="https://colab.research.google.com/github/Hasan-Erbilen/Captcha_project/blob/main/Linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression


def normalise(array):
    """Normalise given array using standard z-score."""
    return (array - array.mean()) / array.std()


def xzero_id(x):
    """Pad x_{1..n} with the multiplicative identity of x_0."""
    return np.c_[np.ones(x.size), x.reshape(-1, 1)]


def linear_model(x, theta):
    """Return theta transpose x."""
    # Set x_0 to 1 for more convenient inner product.
    return xzero_id(x) @ theta


def cost_mse(x, y, theta):
    """Mean squared error cost function."""
    return ((linear_model(x, theta) - y) ** 2).mean()


def gradient_descent(xtrain, ytrain, theta,
                     alpha=1e-2, epsilon=1e-7, iters=256):
    """Return new theta calculated using gradient descent.

    Parameters
    ----------
    xtrain : ndarray
        Training set inputs.
    ytrain : ndarray
        Training set outputs.
    theta : ndarray
        Initial model parameters.
    alpha : float
        Learning rate.
    epsilon : float
        Termination threshold.
    iters : int
        Number of iterations.

    Return
    ------
    theta : ndarray
        New model parameters.
    costs : ndarray
        History of cost per iteration.
    """
    x = xzero_id(xtrain)      # Training set with x_0 = 1
    m = xtrain.size           # Size of training set
    deriv = (-2 * alpha) / m  # Cost derivative vs learning rate
    costs = np.zeros(iters)   # Costs history
    theta = theta.copy()      # Avoid destructive operations

    for i in range(iters):
        delta = deriv * ((linear_model(xtrain, theta) - ytrain) @ x)
        theta += delta
        cost = cost_mse(xtrain, ytrain, theta)

        # Update cost history.  If all deltas exceed threshold,
        # extend cost history and quit.
        if np.all(delta < epsilon):
            costs[i:] = cost
            break
        else:
            costs[i] = cost

    return theta, costs


def main():
    """Print and plot results for assignment."""

    # Read and normalise training data.
    xtrain, ytrain = np.loadtxt('week1.csv', delimiter=',', unpack=True)
    xtrain = normalise(xtrain)
    ytrain = normalise(ytrain)

    # Initial plot of training data.
    plt.figure(constrained_layout=True)
    plt.rc('font', size=16)
    plt.xlabel('Input x (standardised)')
    plt.ylabel('Output y (standardised)')
    plt.scatter(xtrain, ytrain, s=16, c='dimgray', marker='+',
                label='Training data')
    plt.savefig('0-train.pdf')

    # Pick baseline parameters suitable for standardised data.
    base_theta = np.zeros(2)
    base_model = linear_model(xtrain, base_theta)

    # Train model.
    theta, costs = gradient_descent(xtrain, ytrain, base_theta)
    ypred = linear_model(xtrain, theta)
    print('Parameters:', theta)
    print('Cost:', {'baseline': cost_mse(xtrain, ytrain, base_theta),
                    'trained': costs[-1]})

    # Plot against sklearn and baseline models.
    plt.plot(xtrain, ypred, linewidth=3.0, label='Predictions (vanilla)')

    xtrans = xtrain.reshape(-1, 1)
    ypred = LinearRegression().fit(xtrans, ytrain).predict(xtrans)

    plt.plot(xtrain, ypred, ':', linewidth=5.0, label='Predictions (sklearn)')
    plt.plot(xtrain, base_model, '--', linewidth=2.0, label='Baseline')

    plt.legend()
    plt.savefig('1-predict.pdf')
    plt.close()

    # Plot learning rates against cost.
    plt.figure(constrained_layout=True)
    plt.xlabel('Iteration number')
    plt.ylabel('J(θ)')

    for alpha in (1e-3, 1e-2, 1e-1, 9e-1):
        costs = gradient_descent(xtrain, ytrain, base_theta, alpha=alpha)[1]
        plt.plot(costs, linewidth=2.0, label=f'α = {alpha}')

    plt.legend()
    plt.savefig('2-costs.pdf')
    plt.close()


if __name__ == '__main__':
    main()


Parameters: [-1.49182064e-17 -1.89459783e-02]
Cost: {'baseline': 1.0000000000000002, 'trained': 0.964463940888146}
